# Logistic Regression with a Neural Network mindset

## 1 - Packages ##

- [numpy](www.numpy.org) :  for scientific computing with Python.
- [h5py](http://www.h5py.org) : to interact with a dataset that is stored on an H5 file.
- [matplotlib](http://matplotlib.org) : plot graphs in Python.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) : test your model with your own picture

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline